# We Choose to go with Emerald!

In [2]:
# general imports
from math import pi

import networkx as nx

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import Devices

%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [7]:
# set up the Rigetti Ankaa-3 device
device = AwsDevice(Devices.IQM.Emerald)

# list the native gate set
print("The native gates for the", device.name, "device are:")
for gate in device.properties.paradigm.nativeGateSet:
    print(gate)

The native gates for the Emerald device are:
cz
prx
cc_prx
measure_ff


In [15]:
Simple_Identity = Circuit()

Simple_Identity = Simple_Identity.prx(target = 0, angle_1 = pi, angle_2 = 0)
Simple_Identity = Simple_Identity.prx(target = 0, angle_1 = pi, angle_2 = 0)

print(Simple_Identity)

T  : │       0        │       1        │
      ┌──────────────┐ ┌──────────────┐ 
q0 : ─┤ PRx(0, 3.14) ├─┤ PRx(0, 3.14) ├─
      └──────────────┘ └──────────────┘ 
T  : │       0        │       1        │


In [17]:
Spin_Echo_Identity = Circuit()

Spin_Echo_Identity = Spin_Echo_Identity.prx(target = 0, angle_1 = pi/2, angle_2 = 0)
Spin_Echo_Identity = Spin_Echo_Identity.prx(target = 0, angle_1 = pi, angle_2 = pi/2)
Spin_Echo_Identity = Spin_Echo_Identity.prx(target = 0, angle_1 = pi/2, angle_2 = 0)
Spin_Echo_Identity = Spin_Echo_Identity.prx(target = 0, angle_1 = pi, angle_2 = pi/2)

print(Spin_Echo_Identity)

T  : │       0        │         1         │       2        │         3         │
      ┌──────────────┐ ┌─────────────────┐ ┌──────────────┐ ┌─────────────────┐ 
q0 : ─┤ PRx(1.57, 0) ├─┤ PRx(3.14, 1.57) ├─┤ PRx(1.57, 0) ├─┤ PRx(3.14, 1.57) ├─
      └──────────────┘ └─────────────────┘ └──────────────┘ └─────────────────┘ 
T  : │       0        │         1         │       2        │         3         │
